In [1]:
import e3nn_jax as e3nn

In [2]:
def _irreps_from_lmax(
    lmax: int, num_channels: int
) -> e3nn.Irreps:
    """Convenience function to create irreps from lmax."""
    irreps = e3nn.s2_irreps(lmax)
    return (num_channels * irreps).regroup()

In [3]:
irreps = _irreps_from_lmax(5, 64)
keep=e3nn.tensor_product(
        irreps,
        e3nn.Irreps.spherical_harmonics(lmax=5)).regroup()

In [4]:
keep

384x0e+640x1o+320x1e+832x2e+512x2o+896x3o+640x3e+896x4e+640x4o+768x5o+576x5e+576x6e+384x6o+384x7o+256x7e+256x8e+128x8o+128x9o+64x9e+64x10e

In [5]:
irreps.filter(keep=keep)

64x0e+64x1o+64x2e+64x3o+64x4e+64x5o

In [ ]:

irreps.filter(
    keep=e3nn.tensor_product(
        irreps,
        e3nn.Irreps.spherical_harmonics(lmax=5),
    ).regroup()
)